In [48]:
import os
import sys

import numpy as np
import sympy
from matplotlib import pyplot as plt
from sympy import atan2, atanh, cos, sin, sqrt, symbols, tanh, Matrix
from sympy.diffgeom import (BaseCovarDerivativeOp, CoordSystem,
                            CovarDerivativeOp, Manifold, Patch,
                            metric_to_Christoffel_2nd, twoform_to_matrix)
from sympy.diffgeom import TensorProduct as TP
from sympy.utilities import lambdify

location = "/home/gsalinas/GitHub/MyTransport/PyTransport" # This should be the location of the PyTransport folder folder
sys.path.append(location) # Sets up python path to give access to PyTransSetup

import PyTransSetup

PyTransSetup.pathSet()  # This adds the other paths that PyTransport uses to the python path

import PyTransAngular as PyT
import PyTransScripts as PyS

In [29]:
m = Manifold('M', 2)
p = Patch('P', m)

In [76]:
phi, chi = symbols('phi chi', real=True)
r, psi, theta = symbols('r psi theta', real=True, nonnegative=True)
params = symbols('alpha R m_phi', real=True, nonnegative=True)

In [98]:
fields = symbols('r theta', real=True, nonnegative=True)
params = symbols('alpha R m_phi', real=True, nonnegative=True)

f = sympy.symarray('f', len(fields))
p = sympy.symarray('p', len(params))

dict_fields = dict(zip(fields, f))
dict_params = dict(zip(params, p))
dict_combined = {**dict_params, **dict_fields}

In [31]:
relation_dict = {('PhiChi', 'Pol'): [(phi, chi), (sqrt(phi**2 + chi**2), atan2(chi, phi))],
                ('Pol', 'PhiChi'): [(r, theta), (r*cos(theta), r*sin(theta))],
                ('Pol', 'Canoni'): [(r, theta), (sqrt(6*alpha)*atanh(r), theta)],
                ('Canoni', 'Pol'): [(psi, theta), (tanh(psi/sqrt(6*alpha)), theta)]}

In [32]:
R2_phichi = CoordSystem('PhiChi', p, (phi, chi), relation_dict)
R2_polar = CoordSystem('Pol', p, (r, theta), relation_dict)
R2_canonical = CoordSystem('Canoni', p, (psi, theta), relation_dict)

In [41]:
fphi, fchi = R2_phichi.base_scalars()
e_phi, e_chi = R2_phichi.base_vectors()
dphi, dchi = R2_phichi.base_oneforms()

fr, ftheta = R2_polar.base_scalars()
e_r, e_theta = R2_polar.base_vectors()
dr, dtheta = R2_polar.base_oneforms()

fpsi, fthetac = R2_canonical.base_scalars()
e_psi, e_thetac = R2_canonical.base_vectors()
dpsi, dthetac = R2_canonical.base_oneforms()

In [50]:
metric = 6*alpha / (1 - fphi**2 - fchi**2)**2 * (TP(dphi, dphi) + TP(dchi, dchi))
metric

6*alpha*(TensorProduct(dphi, dphi) + TensorProduct(dchi, dchi))/(-phi**2 - chi**2 + 1)**2

In [51]:
twoform_to_matrix(metric)

Matrix([
[6*alpha/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1),                                                                     0],
[                                                                    0, 6*alpha/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)]])

In [52]:
gamma = metric_to_Christoffel_2nd(metric)
gamma

[[[(-4*phi**3 - 4*phi*chi**2 + 4*phi)/(2*(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)), (-4*phi**2*chi - 4*chi**3 + 4*chi)/(2*(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1))], [(-4*phi**2*chi - 4*chi**3 + 4*chi)/(2*(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)), -(-4*phi**3 - 4*phi*chi**2 + 4*phi)/(2*(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1))]], [[-(-4*phi**2*chi - 4*chi**3 + 4*chi)/(2*(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)), (-4*phi**3 - 4*phi*chi**2 + 4*phi)/(2*(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1))], [(-4*phi**3 - 4*phi*chi**2 + 4*phi)/(2*(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)), (-4*phi**2*chi - 4*chi**3 + 4*chi)/(2*(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1))]]]

In [53]:
metric_polar = 6*alpha / (1 - fr**2)**2 * (TP(dr, dr) + fr**2*TP(dtheta, dtheta))
metric_polar

6*alpha*(r**2*TensorProduct(dtheta, dtheta) + TensorProduct(dr, dr))/(1 - r**2)**2

In [102]:
twoform_to_matrix(metric_polar).simplify()

Matrix([
[6*alpha/(r**4 - 2*r**2 + 1),                                0],
[                          0, 6*alpha*r**2/(r**4 - 2*r**2 + 1)]])

In [83]:
str(twoform_to_matrix(metric_polar))

'Matrix([[6*alpha/(r**4 - 2*r**2 + 1), 0], [0, 6*alpha*r**2/(r**4 - 2*r**2 + 1)]])'

In [72]:
import pickle

expr = twoform_to_matrix(metric_polar)
with open("G.txt", "wb") as outf:
    pickle.dump(expr, outf)

In [73]:
with open('G.txt','rb') as f:
    print(pickle.load(f))

Matrix([[6*alpha/(r**4 - 2*r**2 + 1), 0], [0, 6*alpha*r**2/(r**4 - 2*r**2 + 1)]])


In [56]:
gamma_polar = metric_to_Christoffel_2nd(metric_polar)
gamma_polar.simplify()

[[[-2*r/(r**2 - 1), 0], [0, (r**3 + r)/(r**2 - 1)]], [[0, -(r**2 + 1)/(r**3 - r)], [-(r**2 + 1)/(r**3 - r), 0]]]

In [92]:
nF = 2  # number of fields
nP = 3  # number of parameters
f = sympy.symarray('f',nF)   # an array representing the nF fields present for this model [phi, chi]
p = sympy.symarray('p',nP)   # an array representing the nP parameters needed to define this model, format [alpha, R, mphi] 

V = p[0]/2 * p[2]**2 * (f[0]**2 * cos(f[1])**2 + p[1]*f[0]**2 * sin(f[1])**2 )   # this is the potential written in sympy notation
G = 6*p[0]/(1-f[0]**2)**2 * Matrix([[1, 0], [0, 1]]) # this is the field metric written in sympy notation
G

Matrix([
[6*p_0/(1 - f_0**2)**2,                     0],
[                    0, 6*p_0/(1 - f_0**2)**2]])

In [100]:
V = alpha/2 * mphi**2 * r**2 * (cos(theta)**2 + R*sin(theta)**2)
V

alpha*m_phi**2*r**2*(R*sin(theta)**2 + cos(theta)**2)/2

In [101]:
V.subs(dict_combined)

f_0**2*p_0*p_2**2*(p_1*sin(f_1)**2 + cos(f_1)**2)/2

In [93]:
V

p_0*p_2**2*(f_0**2*p_1*sin(f_1)**2 + f_0**2*cos(f_1)**2)/2

In [96]:
dict_params | dict_fields

TypeError: unsupported operand type(s) for |: 'dict' and 'dict'

In [97]:
{**dict_params, **dict_fields}

{alpha: p_0, R: p_1, m_phi: p_2, r: f_0, theta: f_1}